In [ ]:
import pickle
import numpy as np
from collections import defaultdict

from Author import *
from Content import *

## Feed Results to Contents

In [ ]:
start_year, end_year, strategy = 2000, 2019, 'pub'
c_actives = pickle.load(open('content/c_actives.pkl','rb'))
active_ca_adjs = pickle.load(open('content/active_ca_adjs.pkl','rb'))

In [ ]:
c_dists, ca_atts = {}, defaultdict(dict)

for year in range(start_year, end_year):
    c_dist, ca_att = pickle.load(open(f'{strategy}_result/{year}_content_results.pkl','rb'))    
    for c, dist in zip(c_actives[year], c_dist):
        c_dists[c] = dist        
    for (left, right), att in zip(active_ca_adjs[year].T, ca_att):
        ca_atts[left][right] = att

In [ ]:
update_count = 0
author_content_year_utility = defaultdict(dict)

for year in range(start_year, end_year):
    contents = pickle.load(open(f'content/contents_{year}.pkl','rb'))
    for content in contents.values():
        
        if len(content.outcitations)>0:
            update_count += 1
            
            if strategy=='pub':
                content.update_pub_stradis(c_dists[content.id])
                content.update_pub_atts(ca_atts[content.id])
            elif strategy=='cite':
                content.update_cite_stradis(c_dists[content.id])
                content.update_cite_atts(ca_atts[content.id])
            
            author_year_utility = content.split_utility(strategy)
            for author,_,_ in content.authors:
                author_content_year_utility[author][content.id] = author_year_utility[author]
                
    pickle.dump(contents, open('content/contents_{}.pkl'.format(year), 'wb'), -1)
    print(f'Done for year {year}')

pickle.dump(author_content_year_utility, open(f'author/author_content_year_utility_{strategy}.pkl','wb'), -1)
print(update_count, len(c_dists), len(ca_atts)) #2785422

## Feed Results to Authors

In [ ]:
group_names = [2000, 2013, 201700, 201701, 201800, 201801, 
               201900, 201901, 201902, 201903, 201904, 201905, 
               201906, 201907, 201908, 201909, 201910, 201911, 
               201912, 201913, 201914]

In [ ]:
start_year, end_year = 2000, 2019
a_actives = pickle.load(open('author/a_actives.pkl','rb'))
active_ac_adjs = pickle.load(open('author/active_ac_adjs.pkl','rb'))

In [ ]:
a_dists = {'pub':defaultdict(dict), 'cite':defaultdict(dict)}
ac_atts = {'pub':defaultdict(list), 'cite':defaultdict(list)}
aa_alphas = {'pub':defaultdict(dict), 'cite':defaultdict(dict)}
author_content_year_utility = {}

for strategy in ['cite', 'pub']:
    for year in range(start_year, end_year):    
        a_dist, ac_att, aa_alpha = pickle.load(open(f'{strategy}_result/{year}_author_results.pkl','rb'))    
        for a, dist, alpha in zip(a_actives[year], a_dist, aa_alpha):
            a_dists[strategy][a][year] = dist
            aa_alphas[strategy][a][year] = alpha
        for (left, right), att in zip(active_ac_adjs[year].T, ac_att):
            ac_atts[strategy][left].append((year, right, att))
    author_content_year_utility[strategy] = pickle.load(open(f'author/author_content_year_utility_{strategy}.pkl','rb'))

In [ ]:
update_count = 0
author_cite_histories, author_pub_histories = {}, {}

for group_name in group_names:
    
    print('Reading group {}'.format(group_name))
    authors = pickle.load(open('author/authors_{}.pkl'.format(group_name),'rb'))
    
    print('Start for group {}'.format(group_name))
    for author in authors.values():
        
        if len(author.outcitations)>0 and max(author.outcitations)>=2000:
            update_count += 1
                
            author.update_pub_stradiss(a_dists['pub'][author.id])
            author.update_pub_straranks()
            author.update_pub_alphas(aa_alphas['pub'][author.id])
            author.update_pub_atts(ac_atts['pub'][author.id])
            author.update_pub_utilities(author_content_year_utility['pub'][author.id])
            author.update_pub_util_histories()   
            author_pub_histories[author.id] = author.pub_util_histories   
                
            author.update_cite_stradiss(a_dists['cite'][author.id])
            author.update_cite_straranks()
            author.update_cite_alphas(aa_alphas['cite'][author.id])
            author.update_cite_atts(ac_atts['cite'][author.id])
            author.update_cite_utilities(author_content_year_utility['cite'][author.id])
            author.update_cite_util_histories()
            author_cite_histories[author.id] = author.cite_util_histories            
                     
    pickle.dump(authors, open('author/authors_{}.pkl'.format(group_name), 'wb'), -1)
    del authors
    print(f'Done for group {group_name}\n')

pickle.dump(author_cite_histories, open('author/author_cite_histories.pkl','wb'), -1)
pickle.dump(author_pub_histories, open('author/author_pub_histories.pkl','wb'), -1)
print(update_count, len(a_dists['cite']), len(ac_atts['cite']), len(aa_alphas['cite']), len(a_dists['pub']), len(ac_atts['pub']), len(aa_alphas['pub']))